In [ ]:
import os
import json
import torch
import random
import numpy as np
from src.centralized_baseline.dataset import CIFAR100Dataset
from src.centralized_baseline.experiment_manager import ExperimentManager
from src.model_editing.centralized_baseline.trainer import ModelEditingTrainer

from itertools import product

checkpoint_dir = "./checkpoints"
experiments_dir = "./output"

if running_in_colab():
    from google.colab import drive
    drive.mount('/content/drive')

    experiments_dir = "/content/drive/MyDrive/-" # define your Google Drive path here
    checkpoint_dir = experiments_dir + "/checkpoints"


def set_seed(seed):
    """Set random seed for reproducibility"""
    print(f"Setting random seed to {seed}")
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def run_experiments(seed: int):

        set_seed(seed)

        exp = "_EDIT_0"

        grid_dict = {
            "batch_size": [128],
            "lr": [0.01],
            "weight_decay": [5e-4],
            "momentum": [0.9],
            "epochs": [20],
            "seed": [seed],
            "num_workers": [4],
            "accum_steps": [1],
            "optimizer_type": ["SparseSGD"],
            "augment": [None],
            "sparsity": [0.99, 0.95, 0.9], # model editing
            "rounds": [1, 2, 3, 5, 10], # model editing
            "num_batches": [None], # model editing
            "strategy": ["train_least_important"], # model editing
            "approximate_fisher": [True], # model editing
        }

        # Generate param grid from all combinations
        keys, values = zip(*grid_dict.items())
        param_grid = [dict(zip(keys, v)) for v in product(*values)]

        manager = ExperimentManager(
            param_grid=param_grid,
            use_wandb=False,
            project_name="federated-learning-project", #wandb
            group_name="centralized-baseline-model-editing", #wandb
            checkpoint_dir=checkpoint_dir,
        )
        _, _, results = manager.run(
            trainer_class=ModelEditingTrainer,
            dataset_class=CIFAR100Dataset,
            run_name="baseline-model-editing", #wandb
            run_tags=[ f"v{exp}", "baseline-model-editing"], #wandb
            resume_training_from_config=None,
            model_editing=True,  # Enable model editing
        )
        print("Experiments completed.\n")

        filename = f"experiment_baseline_full_param_grid_search_v{exp}_MODEL_EDITING.json"
        os.makedirs(experiments_dir, exist_ok=True)
        file_path = os.path.join(experiments_dir, filename)
        with open(file_path, "w") as f:
            json.dump(results, f, indent=4)
        print(f"Results saved to {file_path}")

try:
    run_experiments(seed=42)
except:
    import traceback
    print(traceback.format_exc())